In [ ]:
'''
Classification - MNIST with CNN model usign TensorFlow
Last Updated : 04/05/2019, by Hyungmin Jun (hyungminjun@outlook.com)

=============================================================================

Copyright 2019 Hyungmin Jun. All rights reserved.

License - GPL version 3
This program is free software: you can redistribute it and/or modify it under
the terms of the GNU General Public License as published by the Free Software
Foundation, either version 3 of the License, or any later version. This
program is distributed in the hope that it will be useful, but WITHOUT ANY
WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR
A PARTICULAR PURPOSE. See the GNU General Public License for more details.
You should have received a copy of the GNU General Public License along with
this program. If not, see <http://www.gnu.org/licenses/>.
'''

In [6]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
from datetime import datetime

mnist = input_data.read_data_sets("../data/MNIST/", one_hot=True)

Extracting ../data/MNIST/train-images-idx3-ubyte.gz
Extracting ../data/MNIST/train-labels-idx1-ubyte.gz
Extracting ../data/MNIST/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST/t10k-labels-idx1-ubyte.gz


In [7]:
print("\ntrain image shape = ", np.shape(mnist.train.images))
print("train label shape = ",   np.shape(mnist.train.labels))
print("test image shape = ",    np.shape(mnist.test.images))
print("test label shape = ",    np.shape(mnist.test.labels))


train image shape =  (55000, 784)
train label shape =  (55000, 10)
test image shape =  (10000, 784)
test label shape =  (10000, 10)


In [9]:
# Hyper parameter
learning_rate = 0.01
epochs        = 30
batch_size    = 100

# Set the placeholder for input and output
X = tf.placeholder(tf.float32, [None, 784])
T = tf.placeholder(tf.float32, [None, 10])

In [16]:
# ==================================================
# Input layer and reshape for convolution
# ==================================================
A1 = X_img = tf.reshape(X, [-1, 28, 28, 1])   # Image size 28 by 28 by 1 (gray image)

# ==================================================
# 1st convolution layer, using 3 by 3 with 32 filters
# ==================================================
F2 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
B2 = tf.Variable(tf.constant(0.1, shape=[32]))

# 28 by 28 by 1 to 28 by 28 by 32
C2 = tf.nn.conv2d(A1, F2, strides=[1, 1, 1, 1], padding='SAME')

# Relu activation function
Z2 = tf.nn.relu(C2 + B2)

# Max pooling; 28 by 28 by 32 to 14 by 14 by 32
A2 = P2 = tf.nn.max_pool(Z2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# ==================================================
# 2nd convolution layer, using 3 by 3 with 64 filters
# ==================================================
F3 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
B3 = tf.Variable(tf.constant(0.1, shape=[64]))

# 14 by 14 by 32 to 14 by 14 by 64
C3 = tf.nn.conv2d(A2, F3, strides=[1, 1, 1, 1], padding='SAME')

# Relu activation function
Z3 = tf.nn.relu(C3 + B3)

# Max pooling; 14 by 14 by 64 to 7 by 7 by 64
A3 = P3 = tf.nn.max_pool(Z3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# ==================================================
# 3rd convolution layer, using 3 by 3 with 128 filters
# ==================================================
F4 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev=0.01))
B4 = tf.Variable(tf.constant(0.1, shape=[128]))

# 7 by 7 by 64 to 7 by 7 by 128
C4 = tf.nn.conv2d(A3, F4, strides=[1, 1, 1, 1], padding='SAME')

# Relu activation function
Z4 = tf.nn.relu(C4 + B4)

# Max pooling; 7 by 7 by 128 to 4 by 4 by 128
A4 = P4 = tf.nn.max_pool(Z4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# ==================================================
# Flatten layer; 4 by 4 with 128 activation maps
# ==================================================
A4_flat = P4_flat = tf.reshape(A4, [-1, 128*4*4])

# ==================================================
# Output layer
# ==================================================
W5 = tf.Variable(tf.random_normal([128*4*4, 10], stddev=0.01))
B5 = tf.Variable(tf.random_normal([10]))

# Linear regression
Z5 = logits = tf.matmul(A4_flat, W5) + B5

Y = A5 = tf.nn.softmax(Z5)

In [19]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Z5, labels=T))
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

# Compare batch_size X by argmax
predicted_val = tf.equal(tf.argmax(A5, 1), tf.argmax(T, 1))

accuracy = tf.reduce_mean(tf.cast(predicted_val, dtype=tf.float32))

In [ ]:
with tf.Session() as sess:
    
    # Initialize tf.Variables
    sess.run(tf.global_variables_initializer())
    
    start_time = datetime.now()
    
    for i in range(epochs):
        
        total_batch = int(mnist.train.num_examples / batch_size)   # 55,000 / 100
        
        for step in range(total_batch):
            
            batch_x_data, batch_t_data = mnist.train.next_batch(batch_size)
            
            loss_val, _ = sess.run([loss, train], feed_dict={X: batch_x_data, T: batch_t_data})
            
            if step % 100 == 0:
                print("epochs = ", i, ", step = ", step, ", loss_val = ", loss_val)
    
    end_time = datetime.now()
    
    print("\nelapsed time = ", end_time - start_time)
    
    # Accuracy
    test_x_data = mnist.test.images     # 10000 * 784
    test_t_data = mnist.test.labels     # 10000 * 10
    
    accuracy_val = sess.run(accuracy, feed_dict={X: test_x_data, T: test_t_data})
    
    print("\nAccuracy = ", accuracy_val)

epochs =  0 , step =  0 , loss_val =  2.8012495
epochs =  0 , step =  100 , loss_val =  0.17730127
epochs =  0 , step =  200 , loss_val =  0.116296805
epochs =  0 , step =  300 , loss_val =  0.07609382
epochs =  0 , step =  400 , loss_val =  0.070986964
epochs =  0 , step =  500 , loss_val =  0.26834992
epochs =  1 , step =  0 , loss_val =  0.118091494
epochs =  1 , step =  100 , loss_val =  0.032817334
epochs =  1 , step =  200 , loss_val =  0.080333225
epochs =  1 , step =  300 , loss_val =  0.033648722
epochs =  1 , step =  400 , loss_val =  0.062489204
epochs =  1 , step =  500 , loss_val =  0.07159407
epochs =  2 , step =  0 , loss_val =  0.010054498
epochs =  2 , step =  100 , loss_val =  0.012050558
epochs =  2 , step =  200 , loss_val =  0.011817139
epochs =  2 , step =  300 , loss_val =  0.008080794
epochs =  2 , step =  400 , loss_val =  0.2861688
epochs =  2 , step =  500 , loss_val =  0.007543987
epochs =  3 , step =  0 , loss_val =  0.06371533
epochs =  3 , step =  100 , l